In [25]:
import pandas as pd
raw_data = pd.read_csv('train_2022.csv')
raw_data

,row_id,TEXT,LABEL
0,0,director dirk shafer and co-writer greg hinton...,0
1,1,"a charming , quirky and leisurely paced scotti...",1
2,2,"the price was good , and came quickly though ...",1
3,3,i was looking forward to this game for a coupl...,0
4,4,arguably the year 's silliest and most incoher...,0
...,...,...,...
1995,1995,an imaginative comedy\/thriller .,1
1996,1996,a savvy exploration of paranoia and insecurity...,1
1997,1997,on the other hand for power grating you ve got...,1
1998,1998,"like dickens with his passages , mcgrath craft...",1


In [26]:
data_augmentation_random_2_words = pd.read_csv('data_augmentation_random_2_words.csv')
data_augmentation_random_2_words

,row_id,TEXT,LABEL
0,0,director dirk broke and co-writer greg hinton ...,0
1,1,"a charming , quirky and leisurely paced scotti...",1
2,2,"the price was my , and came quickly by my prim...",1
3,3,", was great forward to this game for a couple ...",0
4,4,arguably the year 's silliest and most terribl...,0
...,...,...,...
1995,1995,of imaginative comedy\/thriller and,1
1996,1996,"a savvy exploration of paranoia , insecurity i...",1
1997,1997,", the other hand for only grating you ve got s...",1
1998,1998,"like 10-inch with his passages , mcgrath craft...",1


In [27]:
data_augmentation_random_3_words = pd.read_csv('data_augmentation_random_3_words.csv')
data_augmentation_random_3_words

,row_id,TEXT,LABEL
0,0,director stunts shafer and co-writer greg hint...,0
1,1,"a charming , quirky and company paced spy come...",1
2,2,"the price was good , and came you though the p...",1
3,3,", was looking forward to . game for a couple f...",0
4,4,"arguably the year 's half-hearted , to incoher...",0
...,...,...,...
1995,1995,of piece comedy\/thriller of,1
1996,1996,a savvy kind of paranoia and insecurity in ame...,1
1997,1997,", the other hand for power hundred you ve got ...",1
1998,1998,"like 10-inch . his passages , mcgrath crafts q...",1


In [28]:
data_augmentation_chatGPT = pd.read_csv('data_augmentation_chatGPT.csv')
data_augmentation_chatGPT

,TEXT,LABEL,row_id
0,Director Dirk Shafer and co-writer Greg Hinton...,0,0
1,"This Scottish comedy is delightful, quirky, an...",1,1
2,"The price was reasonable, and thanks to my Pri...",1,2
3,I had been eagerly anticipating this game for ...,0,3
4,"Arguably, this is the silliest and most confus...",0,4
...,...,...,...
1995,A creative comedy/thriller.,1,1995
1996,Explores paranoia and insecurity in America's ...,1,1996
1997,Good for power grating.,1,1997
1998,McGrath's variation on the novel crafts moving...,1,1998


In [29]:
merged_data = pd.concat([raw_data, data_augmentation_random_2_words, data_augmentation_random_3_words, data_augmentation_chatGPT], ignore_index=True)
merged_data

,row_id,TEXT,LABEL
0,0,director dirk shafer and co-writer greg hinton...,0
1,1,"a charming , quirky and leisurely paced scotti...",1
2,2,"the price was good , and came quickly though ...",1
3,3,i was looking forward to this game for a coupl...,0
4,4,arguably the year 's silliest and most incoher...,0
...,...,...,...
7995,1995,A creative comedy/thriller.,1
7996,1996,Explores paranoia and insecurity in America's ...,1
7997,1997,Good for power grating.,1
7998,1998,McGrath's variation on the novel crafts moving...,1


In [32]:
final_train_data = merged_data.copy()
final_train_data['LABEL'] = final_train_data['LABEL'].replace(1, 2)
final_train_data

,row_id,TEXT,LABEL
0,0,director dirk shafer and co-writer greg hinton...,0
1,1,"a charming , quirky and leisurely paced scotti...",2
2,2,"the price was good , and came quickly though ...",2
3,3,i was looking forward to this game for a coupl...,0
4,4,arguably the year 's silliest and most incoher...,0
...,...,...,...
7995,1995,A creative comedy/thriller.,2
7996,1996,Explores paranoia and insecurity in America's ...,2
7997,1997,Good for power grating.,2
7998,1998,McGrath's variation on the novel crafts moving...,2


In [33]:
%%time

from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence

# 检查 CUDA 是否可用并设置设备
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
print(f"Using device: {device}")

# 加载预训练的 RoBERTa tokenizer 和模型
tokenizer = RobertaTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

# 将模型转移到指定的设备
model = model.to(device)

# 假设 raw_data 是你的 DataFrame 并已经准备好
train_data = final_train_data.copy()

# 使用 tokenizer 将文本转换为 token IDs
def tokenize_text(text):
    return tokenizer(text, return_tensors='pt', padding=True, truncation=True)['input_ids'].squeeze()

train_data['input_ids'] = train_data['TEXT'].apply(tokenize_text)

# 转换成可以输入模型的格式
inputs = pad_sequence(train_data['input_ids'].tolist(), batch_first=True).to(device)

# 将数据拆分为训练集和测试集
train_inputs, test_inputs, train_labels, test_labels = train_test_split(inputs, train_data['LABEL'].tolist(), test_size=0.2)

# 将标签也转移到 CUDA
train_labels = torch.tensor(train_labels).to(device)
test_labels = torch.tensor(test_labels).to(device)

# 创建 PyTorch DataLoader
train_dataset = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# 定义 optimizer 和损失函数
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# 训练模型
model.train()
for epoch in range(5):
    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

# 评估模型
model.eval()
with torch.no_grad():
    outputs = model(test_inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# 将预测结果从 CUDA 转移到 CPU 并转换为 NumPy 数组
predictions = predictions.cpu().numpy()

# 轉換格式
modified_predictions = np.where(predictions == 2, 1, predictions)
modified_test_labels = np.where(test_labels.cpu() == 2, 1, test_labels.cpu())
# 打印预测结果
report = classification_report(modified_test_labels, modified_predictions)  # 確保標籤也在 CPU 上
print(report)

Using device: cpu


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


              precision    recall  f1-score   support

           0       0.94      0.88      0.91       834
           1       0.88      0.94      0.91       766

    accuracy                           0.91      1600
   macro avg       0.91      0.91      0.91      1600
weighted avg       0.91      0.91      0.91      1600

CPU times: total: 13h 42min 40s
Wall time: 4h 46min 21s


In [34]:
import datetime
import pytz
import pandas as pd
def export_csv(df,name):
  now = datetime.datetime.now().astimezone(pytz.timezone('Asia/Taipei'))
  formatted_time = now.strftime('%Y%m%d')
  df.to_csv('result/'+ formatted_time + '_' + name + ".csv", index=False,encoding="utf_8_sig")

In [35]:
%%time
import pandas as pd
import torch
from transformers import RobertaTokenizer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset

# 加载测试数据
test_data = pd.read_csv('test_no_answer_2022.csv')
# 使用与训练数据相同的Tokenizer处理文本
def tokenize_text(text):
    return tokenizer(text, return_tensors='pt', padding='max_length', max_length=512, truncation=True)['input_ids'].squeeze()

test_data['input_ids'] = test_data['TEXT'].apply(tokenize_text)

# 将处理后的文本转换为模型可以接收的格式
test_inputs = pad_sequence(test_data['input_ids'].tolist(), batch_first=True).to(device)

# 创建 DataLoader
test_dataset = TensorDataset(test_inputs)
test_loader = DataLoader(test_dataset, batch_size=8)  # 可调整batch size

# 使用模型进行预测
model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        inputs = batch[0]
        outputs = model(inputs)
        pred = torch.argmax(outputs.logits, dim=-1)
        predictions.extend(pred.cpu().numpy())

# 转换格式，将类别2映射为1，如果需要
predictions = np.where(np.array(predictions) == 2, 1, np.array(predictions))

# 显示或保存预测结果
print(predictions)

# 可选择将预测结果保存到CSV文件
results = pd.DataFrame({
    "row_id": test_data['row_id'],
    "prediction": predictions
})
export_csv(results,'twitter')

[1 1 0 ... 0 1 0]
CPU times: total: 14h 20min 50s
Wall time: 4h 29min 15s


In [5]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from datasets import Dataset

# 加載數據集

raw_data['LABEL'] = raw_data['LABEL'].map({0: 'negative', 1: 'positive'})  # 將標籤轉換為文字

# 載入預訓練模型和分詞器
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3, return_dict=True)

# 數據集切分
train_df, eval_df = train_test_split(raw_data, test_size=0.1)

# 將 DataFrame 轉換為 Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

# 定義數據預處理函數
def preprocess_function(examples):
    return tokenizer(examples['TEXT'], truncation=True, padding='max_length', max_length=128), {'labels': examples['LABEL']}

# 對數據集應用預處理
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]


TypeError: Provided `function` which is applied to all elements of table returns a variable of type <class 'tuple'>. Make sure provided `function` returns a variable of type `dict` (or a pyarrow table) to update the dataset or `None` if you are only interested in side effects.

In [4]:
training_args = TrainingArguments(
    output_dir='./results',           # 輸出目錄
    num_train_epochs=3,               # 訓練的總輪數
    per_device_train_batch_size=8,    # 訓練的 batch size
    per_device_eval_batch_size=16,    # 驗證的 batch size
    warmup_steps=500,                 # 預熱步驟
    weight_decay=0.01,                # 權重衰減
    logging_dir='./logs',             # 日誌目錄
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()  # 開始訓練

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
import numpy as np

# 使用訓練好的模型進行預測
predictions = trainer.predict(eval_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=-1)

# 打印分類報告
report = classification_report(eval_dataset['LABEL'], predicted_labels, target_names=['negative', 'neutral', 'positive'])
print(report)